In [14]:
import pandas as pd
import numpy as np
import tia.bbg.datamgr as dm
mgr=dm.BbgDataManager()

#use 'Book1.xlsx' as the example
print("use 'Book1.xlsx' as the example")
files=input()
df = pd.read_excel(files)
mgr.sid_result_mode = 'frame'

c=0
empty=[]
for x in df['CUSIP']:
    empty.append('/cusip/'+str(df.at[c,'CUSIP']))
    c=c+1
    next
cats=['PX_LAST','PAR_AMT','CPN','CPN_FREQ','DAYS_TO_NEXT_COUPON','DAYS_TO_MTY']
liss=[]
for b in empty:
    store=mgr[b]
    first=store['PX_LAST','PAR_AMT','CPN','CPN_FREQ','DAYS_TO_NEXT_COUPON','DAYS_TO_MTY']
    liss.append(first)
g=2

dfdf=liss[0]
dfdf=liss[0].append(liss[1])
for i in range(2,len(liss)):
    dfdf=dfdf.append(liss[g])
    g=g+1

def Build_info(x):    
    dfdf['COUPON_PAYMENT']=(dfdf['CPN']*dfdf['PAR_AMT'])/100
    dfdf['DTM_DNC']=dfdf['DAYS_TO_MTY']-dfdf['DAYS_TO_NEXT_COUPON']
    dfdf['YR_FREQ']=365/dfdf['CPN_FREQ']
    dfdf['#_OF_CPN']=np.rint(dfdf['DTM_DNC']/dfdf['YR_FREQ'])
    return;

def Duration_calc(x):
        
    for index, row in dfdf.iterrows():
        #other
        ation=((row['DAYS_TO_NEXT_COUPON']*row['COUPON_PAYMENT'])+(((row['COUPON_PAYMENT']+row['PAR_AMT'])*row['DAYS_TO_MTY']))/row['PX_LAST'])
    
        #final
        dur=(((row['COUPON_PAYMENT']+row['PAR_AMT'])*row['DAYS_TO_MTY'])/row['PX_LAST'])
        if row['DTM_DNC']>row['YR_FREQ']:
            #normal
            store_val=0
            for x in range(row['#_OF_CPN'].astype(np.int64)):
                sols=0
                sols=row['COUPON_PAYMENT']*(row['DAYS_TO_NEXT_COUPON']+(row['YR_FREQ']*x))
                store_val=store_val+sols
            dfdf.at[index,'DURATION']="{:10.0f}".format(ation+store_val)
        if row['DTM_DNC']<=row['YR_FREQ']:
            if row['DAYS_TO_NEXT_COUPON']==row['DAYS_TO_MTY']:
                #final
                dfdf.at[index,'DURATION']="{:10.0f}".format(dur)
            if row['DAYS_TO_MTY']>row['YR_FREQ']:
                #other
                dfdf.at[index,'DURATION']="{:10.0f}".format(ation)
    return;       



use 'Book1.xlsx' as the example
'Book1.xlsx'
